In [1]:
import pandas as pd
import numpy as np
import re
import string

%matplotlib notebook
import matplotlib.pyplot as plt

Hago una limpieza básica de los reviews descargados:

In [2]:
# # Leo los datos:
# rs = np.random.RandomState(127361824)
# # list_parts = np.array(os.listdir('./parts/')) 
# # num_parts = len(list_parts)
# # df = pd.concat([pd.read_csv('./parts/{}'.format(filename),lineterminator='\n') for filename in rs.choice(list_parts,num_parts,replace=False)])

# # list_parts = np.array(os.listdir('../29-mercado-libre-api-v3/parts/')) 
# # num_parts = len(list_parts)
# # dfs_list = []
# # for filename in rs.choice(list_parts,num_parts,replace=False):
# #     try:
# #         df = pd.read_csv('../29-mercado-libre-api-v3/parts/{}'.format(filename))
# #         dfs_list.append(df)
# #     except:
# #         print(filename)

# # df = pd.concat(dfs_list)

# list_parts1 = np.array(os.listdir('../29-mercado-libre-api-v3/parts/')) 
# num_parts1 = len(list_parts1)
# df1 = pd.concat([pd.read_csv('../29-mercado-libre-api-v3/parts/{}'.format(filename),lineterminator='\n') for filename in rs.choice(list_parts1,num_parts1,replace=False)])

# list_parts2 = np.array(os.listdir('./parts/')) 
# num_parts2 = len(list_parts2)
# df2 = pd.concat([pd.read_csv('./parts/{}'.format(filename),lineterminator='\n') for filename in rs.choice(list_parts2,num_parts2,replace=False)])

# df = pd.concat([df1, df2])

# print('Cantidad de reviews descargados:',len(df))
# df.head(5)

Cantidad de reviews descargados: 14594217


,prod_id,cat_id,review_id,country,prod_title,reviewer_id,review_date,review_status,review_title,review_content,review_rate,review_likes,review_dislikes
0,MLA882956549,MLA10200,37332886,MLA,"Alquiler Inflables Zona Norte, Pelotero, Plaza...",13116850,2019-01-02T12:42:00.000-04:00,published,Excelente,"Súper puntuales, material de primera calidad, ...",5,3,0
1,MLA882956549,MLA10200,36795845,MLA,"Alquiler Inflables Zona Norte, Pelotero, Plaza...",23888292,2018-12-15T20:53:14.000-04:00,published,"Excelente, bebés y chicos felices",Excelente y rápida atención desde el principio...,5,3,0
2,MLA882956549,MLA10200,21871450,MLA,"Alquiler Inflables Zona Norte, Pelotero, Plaza...",66883442,2018-01-12T00:26:28.000-04:00,published,"Optimo, muy recomendable",El material es de muy buena calidady los chico...,5,2,0
3,MLA882956549,MLA10200,26445411,MLA,"Alquiler Inflables Zona Norte, Pelotero, Plaza...",239377732,2018-03-28T13:44:52.000-04:00,published,Muy bueno,"Servicio de calidad, puntuales, y los inflable...",5,2,0
4,MLA882956549,MLA10200,21665685,MLA,"Alquiler Inflables Zona Norte, Pelotero, Plaza...",10984535,2018-01-09T15:54:40.000-04:00,published,super recomendable,Llegaron a tiempo y el castillo que trajeron n...,5,2,0


In [3]:
# # Elimino los duplicados:
# df['review_id'] = df['review_id'].astype(np.int)
# df = df.drop_duplicates(subset=['review_id']).reset_index(drop=True)
# print('Cantidad de reviews únicos descargados:',len(df))

Cantidad de reviews únicos descargados: 7874526


In [2]:
#df.to_csv('./unique_reviews_24-12-2020.csv',index=False)
df = pd.read_csv('./unique_reviews_24-12-2020.csv')
print('Cantidad de reviews únicos descargados:',len(df))

Cantidad de reviews únicos descargados: 7899951


In [3]:
# Elimino los que se descargaron mal por algún motivo
df = df[df['country'].isin(['MLB', 'MLA', 'MLM', 'MLU', 'MCO', 'MLC', 'MLV', 'MPE'])].reset_index(drop=True)
df = df[df['review_rate'].isin([1., 2., 3., 4., 5.])].reset_index(drop=True)
df = df[df['review_title'].apply(type).eq(str)].reset_index(drop=True)
df = df[df['review_content'].apply(type).eq(str)].reset_index(drop=True)
print('Cantidad de reviews únicos y correctamente descargados:',len(df))

Cantidad de reviews únicos y correctamente descargados: 7873143


In [4]:
# Cambio todos los espacios por espacios simples y vuelvo a eliminar duplicados
df['review_content'] = df['review_content'].astype(np.str).str.replace(r'\s+',' ',regex=True)
df['review_title'] = df['review_title'].astype(np.str).str.replace(r'\s+',' ',regex=True)
df['review_rate'] = df['review_rate'].astype(np.int)
df = df.drop_duplicates(subset=['review_content','review_title','review_rate']).reset_index(drop=True)
print('Cantidad de reviews con contenido, título y rate únicos:',len(df))

# Dataframe con país, contenido, título y rate únicamente:
df['country'] = df['country'].astype(np.str)
df = df.loc[:,['country','review_content','review_title','review_rate']]
df.head(5)

Cantidad de reviews con contenido, título y rate únicos: 7311556


,country,review_content,review_title,review_rate
0,MLA,"Súper puntuales, material de primera calidad, ...",Excelente,5
1,MLA,Excelente y rápida atención desde el principio...,"Excelente, bebés y chicos felices",5
2,MLA,El material es de muy buena calidady los chico...,"Optimo, muy recomendable",5
3,MLA,"Servicio de calidad, puntuales, y los inflable...",Muy bueno,5
4,MLA,Llegaron a tiempo y el castillo que trajeron n...,super recomendable,5


**NOTA IMPORTANTE: FALTA PONER UNA COTA POR PRODUCTO (ES DECIR, LIMITAR LA CANTIDAD MÁXIMA DE REVIEWS POR PRODUCTO) ANTES DE REDUCIR LOS DATOS BAJADOS AL CONTENIDO, TÍTULO Y RATE**

In [8]:
# Genero un programa para eliminar los reviews que están en otro idioma:
# NOTA: NO DESCOMENTAR. TARDA MUCHO EN CORRER EL SCRIPT
# df.to_csv('./reviews_full_raw.csv',index=False)
# !python lang_detect.py

df = pd.read_csv('./reviews_full_raw.csv')
mask_spacy_es = pd.read_csv('./mask_spacy_es.csv',squeeze=True)
# mask_spacy_pt = pd.read_csv('./mask_spacy_pt.csv',squeeze=True)
# mask_langdetect = pd.read_csv('./mask_langdetect.csv',squeeze=True)
# mask_langid = pd.read_csv('./mask_langid.csv',squeeze=True)
# mask_fasttext = pd.read_csv('./mask_fasttext.csv',squeeze=True)
mask = mask_spacy_es #& mask_spacy_pt & mask_langdetect & mask_langid & mask_fasttext
df = df[mask].reset_index(drop=True)

In [9]:
from collections import Counter
chars_vocab = dict(Counter((df['review_content'] + df['review_title']).str.cat(sep='')))
idx = np.argsort(list(chars_vocab.values()))[::-1]
chars_vocab = {list(chars_vocab.keys())[i]:list(chars_vocab.values())[i] for i in idx}
chars_vocab

{' ': 50525427,
 'e': 37894001,
 'o': 28394694,
 'a': 27221960,
 'n': 17499418,
 'r': 15572056,
 'i': 15533243,
 't': 15120601,
 'u': 15033729,
 'c': 14615010,
 's': 13384402,
 'l': 13108490,
 'd': 12964260,
 'm': 11167065,
 'p': 8565006,
 '.': 6388381,
 'b': 5465403,
 'y': 3612265,
 ',': 3139345,
 'x': 2724694,
 'E': 2718257,
 'f': 2591888,
 'q': 2274213,
 'v': 2027611,
 'g': 1948443,
 'M': 1881410,
 '!': 1731769,
 'h': 1671438,
 'ó': 984525,
 'B': 933847,
 'á': 912691,
 'P': 736890,
 'j': 720573,
 'é': 588883,
 'ã': 483034,
 'í': 469658,
 'z': 462401,
 'A': 407263,
 'L': 406578,
 'S': 404953,
 'R': 392817,
 '0': 383108,
 'C': 362545,
 'T': 338554,
 'O': 303112,
 'N': 281507,
 'ç': 280949,
 '1': 235206,
 'ñ': 214117,
 'Ó': 208230,
 'F': 192006,
 'ú': 190696,
 'G': 179802,
 'D': 165390,
 'U': 131020,
 'k': 128789,
 'I': 125292,
 '%': 101913,
 '?': 85457,
 'V': 84736,
 'H': 81340,
 '2': 81303,
 '-': 71194,
 '5': 58146,
 '3': 57477,
 ')': 55412,
 'Y': 51913,
 'w': 48558,
 'Q': 47031,
 'ê

In [12]:
df[(df['review_content']+df['review_title']).str.contains('¥',regex=False)]

#* (df['country'] != 'MLB')]

,country,review_content,review_title,review_rate
944966,MLB,Horrível. Não pode amarrar dos dois lados e ti...,o que posso achar de um produto desse nível?#%...,1
971274,MLB,%©®®™?[]{}=°^¥€¢£~`|•??÷×¶?\.,Muito bom,4
1100567,MLM,*mu¥* buena calidad *de* producto *grasias*??.,una muy buena iluminación y me gusto mucho bien,5


Separo por idioma los reviews. Para eso, primero separo por país (en todo lo que es Brasil y lo que no es Brasil) y después me fijo si alguno de los vocabularios contiene palabras que no son del idioma que le corresponde. Si este fuera el caso, voy a buscar el review en el que aparece la palabra y si está en otro idioma.

In [5]:
# Cuento las muestras por país
print('Muestras por país:')
print(df['country'].value_counts())
print()

# Cuento las muestras por idioma, a partir del país
print('Muestras por idioma:')
df_por = df[df['country'] == 'MLB'].reset_index(drop=True)
df_esp = df[df['country'] != 'MLB'].reset_index(drop=True)
print('Portugués:', len(df_por))
print('Español:', len(df_esp))
print()

# Obtengo los vocabularios:
print('Vocabulario de palabras en español:')
splitted_esp = (df_esp['review_content'] + df_esp['review_title']).str.lower().str.split(r'[0-9 {}]+'.format(re.escape(string.punctuation)))
vocab_esp = splitted_esp.explode().value_counts()
print(vocab_esp)
print()

print('Vocabulario de palabras en portugués:')
splitted_por = (df_por['review_content'] + df_por['review_title']).str.lower().str.split(r'[0-9 {}]+'.format(re.escape(string.punctuation)))
vocab_por = splitted_por.explode().value_counts()
print(vocab_por)

with open('./words_esp_full.txt','w') as file:
    for word in vocab_esp.index:
        file.write(word)
        file.write('\n')

with open('./words_por_full.txt','w') as file:
    for word in vocab_por.index:
        file.write(word)
        file.write('\n')

Muestras por país:
MLB    2399883
MLA    2124356
MLM    1525680
MCO     454351
MLC     364761
MLU     325120
MLV      83270
MPE      34135
Name: country, dtype: int64

Muestras por idioma:
Portugués: 2399883
Español: 4911673

Vocabulario de palabras en español:


KeyboardInterrupt: 

In [7]:
df_por['review_rate'].value_counts()

5    1801394
4     324174
3     130169
1      88651
2      55495
Name: review_rate, dtype: int64

In [23]:
countries = ['MLA','MLM','MLU','MCO','MLC','MLV','MPE']
rates = np.arange(5)+1
matrix = np.zeros((len(countries),len(rates)),dtype=np.int)
for i,country in enumerate(countries):
    for j,rate in enumerate(rates):
        mymask = (df_esp['review_rate'] == rate) & (df_esp['country'] == country)
        matrix[i,j] = len(df_esp[mymask])

print(matrix)

[[ 19773  17658  62912 205802 741560]
 [ 21226  13431  36958 116324 636250]
 [  5146   3634  13161  43404 167505]
 [  6166   4364  10302  35537 172633]
 [  6041   3544   7557  17807 120825]
 [  1117    609   2859   8215  45449]
 [   670    368   1304   4000  17208]]


In [6]:
countries = ['MLA','MLM','MLU','MCO','MLC','MLV','MPE']
rates = np.arange(5)+1
matrix = np.zeros((len(countries),len(rates)),dtype=np.int)
for i,country in enumerate(countries):
    for j,rate in enumerate(rates):
        mymask = (df_esp['review_rate'] == rate) & (df_esp['country'] == country)
        matrix[i,j] = len(df_esp[mymask])

print(matrix)

[[  46618   38504  128580  407944 1502710]
 [  44132   26385   68360  209257 1177546]
 [   7634    5297   18495   59676  234018]
 [  13159    9024   20402   68681  343085]
 [  16056    8828   17801   40526  281550]
 [   1671     889    4025   11278   65407]
 [    997     504    1843    5720   25071]]


In [2]:
# # Cuento las muestras por país
# print('Muestras por país:')
# print(df['country'].value_counts())
# print()

# # Cuento las muestras por idioma, a partir del país
# print('Muestras por idioma:')
# df_por = df[df['country'] == 'MLB'].reset_index(drop=True)
# df_esp = df[df['country'] != 'MLB'].reset_index(drop=True)
# print('Portugués:', len(df_por))
# print('Español:', len(df_esp))

# df_esp.loc[:,['review_content','review_rate']].to_csv('./data_esp_full.csv',index=False)
# df_por.loc[:,['review_content','review_rate']].to_csv('./data_por_full.csv',index=False)

# df_esp = pd.read_csv('./data_esp_full.csv')
# df_por = pd.read_csv('./data_por_full.csv')

# print('Muestras por idioma:')
# print('Portugués:', len(df_por))
# print('Español:', len(df_esp))

# df_esp.head(5)

Muestras por idioma:
Portugués: 1540355
Español: 2879234


,review_content,review_rate
0,Todo ok. Sin ningún inconveniente.,5.0
1,"Bueno hasta ahorita, solo la calidad del plást...",4.0
2,Perfecto repuesto para una tornamesa en restau...,5.0
3,"Excelente aguja, funciona muy bien en mi torna...",5.0
4,"Muy buenas agujas, el desempeño es casi igual ...",5.0
